In [1]:
project      = 'hoshinolab_proteomics_age'
from bin import nimb_link
NIMB_HOME = nimb_link.link_with_nimb()
from setup.get_vars import Get_Vars, SetProject
from distribution import utilities as utils
from stats.db_processing import Table
from stats.preprocessing import Preprocess
from stats import preprocessing, varia
from stats.make_stats_grid import MakeGrid
from stats.stats_models import ANOVA_do
from stats.plotting import Make_Plot_Regression, Make_plot_group_difference
tab = Table()

all_vars = Get_Vars()
project_vars = all_vars.projects[project]

getvars      = Get_Vars()
nimb_stats   = getvars.stats_vars
projects     = getvars.projects
NIMB_tmp     = getvars.location_vars['local']['NIMB_PATHS']['NIMB_tmp']
fname_groups = projects[project]["fname_groups"]
nimb_stats   = SetProject(NIMB_tmp, nimb_stats, project, fname_groups).stats
project_vars = projects[project]
stats_paths  = nimb_stats['STATS_PATHS']

def get_X_data_per_group_all_groups(group):
    # extract X_scaled values for the brain parameters
        predicted_target = project_vars["prediction_target"]
        if not predicted_target:
            predicted_target = project_vars["group_col"]
        if group == 'all':
                df_clin_group = df_user_stats
                df_X          = df_adjusted
                y_labeled     = preprocessing.label_y(df_user_stats, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        else:
                df_group      = tab.get_df_per_parameter(df_final_grid, project_vars['group_col'], group)
                df_clin_group = tab.rm_cols_from_df(df_group, cols_X)
                df_X          = tab.rm_cols_from_df(df_group, [i for i in df_group.columns.tolist() if i not in cols_X])
                y_labeled     = preprocessing.label_y(df_group, predicted_target)
                X_scaled      = preprocessing.scale_X(df_X)
        return df_X, y_labeled, X_scaled, df_clin_group

/home/ssp/Dropbox/github/nimb/nimb
        credentials are located at: /home/ssp/nimb
        credentials are located at: /home/ssp/nimb


In [7]:
#project_vars

In [2]:
df_user_stats, df_final_grid,\
            df_adjusted,\
            cols_X,\
            groups = MakeGrid(project_vars,
                                nimb_stats).grid()

    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/stats_src.xlsx,
        sheet: 0
    reading file: /media/g/nimb_tmp/projects/hoshinolab_proteomics_age/stats_src/stats/grid.csv,
        sheet: 0


In [12]:
df_final_grid.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
for col in df_final_grid.columns:
    if df_final_grid[col].any(0):
            print(col, )

A6NNI4
True
CON_00708398_1
True
CON_LysC
True
CON_Trypsin
True
O43866
True
P00450
True
P00488
True
P00739
True
P01008
True
P01009
True
P01011
True
P01023
True
P01024
True
P01031
True
P01593
True
P01597
True
P01625
True
P01764
True
P01767
True
P01781
True
P01833
True
P01859
True
P01876
True
P02042
True
P02647
True
P02652
True
P02675
True
P02745
True
P02760
True
P02763
True
P02765
True
P02766
True
P02768
True
P02775
True
P02776
True
P02787
True
P02790
True
P04003
True
P04114
True
P04196
True
P04220
True
P04275
True
P08603
True
P13671
True
P19652
True
P22792
True
P26038
True
P35579
True
P60709
True
P61981
True
P68871
True
P69905
True
Q08380
True
age
True
group
adult


Index(['control1_mom', 'control2_dad', 'control_dad_Jon', 'control_mom_HZ',
       'AH', 'DG', 'FS', 'SS', 'EA', 'EG', 'JM', 'CTRL_11', 'CTRL_14',
       'CTRL_16', 'CTRL_17', 'CTRL_44', 'CTRL_49', 'CTRL_50', 'CTRL_53',
       'CTRL_63', 'CTRL_64', 'CTRL_1_MOM', 'CTRL_11_MOM', 'CTRL_15_MOM',
       'CTRL_17_MOM', 'CTRL_42_MOM', 'CTRL_44_MOM', 'CTRL_49_MOM',
       'CTRL_58_MOM', 'CTRL_63_MOM', 'CTRL_64_MOM', 'CTRL_14_DAD',
       'CTRL_16_DAD', 'CTRL_17_DAD', 'CTRL_42_DAD', 'CTRL_46_DAD',
       'CTRL_48_DAD', 'CTRL_50_DAD', 'CTRL_53_DAD', 'CTRL_63_DAD',
       'CTRL_64_DAD', 'PAT_5_MOM', 'PAT_9_MOM', 'PAT_11_MOM', 'PAT_12_MOM',
       'PAT_23_MOM', 'PAT_25_MOM', 'PAT_26_MOM', 'PAT_27_MOM', 'PAT_28_MOM',
       'PAT_34_MOM', 'PAT_5_DAD', 'PAT_9_DAD', 'PAT_11_DAD', 'PAT_12_DAD',
       'PAT_23_DAD', 'PAT_24_DAD', 'PAT_25_DAD', 'PAT_26_DAD', 'PAT_28_DAD',
       'PAT_34_DAD'],
      dtype='object', name='sample name')

In [ ]:
preproc = Preprocess(utils)


In [5]:
group = 'all'
df_X, y_labeled, X_scaled, df_clin_group = get_X_data_per_group_all_groups(group)
features = cols_X

PowerTransformer()


/home/ssp/.local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3234: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [26]:
sig_cols = ANOVA_do(df_final_grid,
                    project_vars['variables_for_glm'], features,
                    varia.get_dir(stats_paths['anova']),
                    p_thresh = 0.05, intercept_thresh = 0.05).sig_cols

    analysing 55 features for parameter: age


PatsyError: model is missing required outcome variables

In [6]:
Make_Plot_Regression(df_final_grid,
                    sig_cols, project_vars['group_col'],
                    varia.get_dir(stats_paths['simp_lin_reg_dir']))

In [25]:
Make_plot_group_difference(df_final_grid,
                            sig_cols, project_vars['group_col'], groups,
                            varia.get_dir(stats_paths['anova']))